In [62]:
import sys,os
import uproot4 as uproot
import awkward1 as ak
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import kinematics
from particle import literals as lp
from fcc_python_tools import plotting
import tensorflow as tf
import zfit
import random

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

file_type = "p8_ee_Ztautau_ecm91_EvtGen_Tau2MuGamma"

#Load 1 sub-file of 100k events
file = uproot.open(f"/eos/experiment/fcc/ee/generation/DelphesEvents/fcc_tmp/{file_type}/events_001720714.root")
tree = file['events']

#Get all the variables in the tree
vars = tree.keys()

#Loacation of the reco particles
r_container = "ReconstructedParticles"
r_c = f'{r_container}/{r_container}'

#Keep the variables that aren't covMatrix
keep_vars = []
for v in vars:
    if("covMatrix" not in v and r_c in v):
        keep_vars.append(v)

r = tree.arrays(keep_vars,how="zip")

In [63]:
r[r_c,'p'] = kinematics.calc_p(r,r_c)
p_cut = r[r_c,"p"] > 1.
r = r[p_cut]

#Muons
mu_cut = abs(r[r_c,"mass"] - lp.mu_plus.mass/1000.) < 1e-4
r["mu"] = r[r_c][mu_cut]

#Photons
gamma_cut = abs(r[r_c,"mass"] - lp.gamma.mass/1000.) < 1e-4
r["gamma"] = r[r_c][gamma_cut]

In [64]:
tau = ak.cartesian({"mu": r["mu"], "gamma": r["gamma"]})

PDG_mu_m = lp.mu_plus.mass/1000.
PDG_gamma_m = lp.gamma.mass/1000.
mass = kinematics.mass([tau["mu"], tau["gamma"]], [PDG_mu_m, PDG_gamma_m])

In [65]:
PDG_tau_m = lp.tau_plus.mass/1000.
window = 0.6
mass_cut = abs(mass - PDG_tau_m) < window
mass = mass[mass_cut]

In [66]:
low = PDG_tau_m - window
high = PDG_tau_m + window
obs = zfit.Space('mtau', limits=(low, high))

#Signal PDF
rand = random.randint(0,9999)
mu = zfit.Parameter(f"mu_{rand}", 1.8, low, high)
sigmaL = zfit.Parameter(f"sigmaL_{rand}", 0.1, 0., 0.5)
sigmaR = zfit.Parameter(f"sigmaR_{rand}", 0.05, 0., 0.5)
alphaL = zfit.Parameter(f"alphaL_{rand}", 1., 0., 5.)
nL = zfit.Parameter(f"nL_{rand}", 100., 0., 500.)
alphaR = zfit.Parameter(f"alphaR_{rand}", -1., -5., 0.)
nR = zfit.Parameter(f"nR_{rand}", 100., 0., 500.)
frac = zfit.Parameter(f"frac_{rand}", 0.3, 0., 1.)

pdf_sigL = zfit.pdf.CrystalBall(obs=obs, mu=mu, sigma=sigmaL, alpha=alphaL, n=nL)
pdf_sigR = zfit.pdf.CrystalBall(obs=obs, mu=mu, sigma=sigmaR, alpha=alphaR, n=nR)
pdf = zfit.pdf.SumPDF([pdf_sigL, pdf_sigR], frac)

In [67]:
data_np = ak.to_numpy(ak.flatten(mass))[0:10000]
data = zfit.Data.from_numpy(obs=obs, array=data_np)

In [68]:
nll = zfit.loss.UnbinnedNLL(model=pdf, data=data)
minimizer = zfit.minimize.Minuit(tolerance=1e-5)
result = minimizer.minimize(nll)
param_errors, _ = result.errors(method="minuit_minos")

print("m(tau) fit function minimum:", result.fmin)
print("m(tau) fit converged:", result.converged)
print("m(tau) fit full minimizer information:", result.info)

params = result.params
print(params)

┌──────────────────────────────────┬──────────────────────────────────────┐
│ FCN = -5278                      │      Ncalls = 1145 (1145 total)      │
│ EDM = 3.53e-07 (Goal: 1e-05)     │               up = 0.5               │
├───────────────┬──────────────────┼──────────────────────────────────────┤
│ Valid Minimum │ Valid Parameters │       SOME Parameters at limit       │
├───────────────┴──────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│   Hesse ok    │  Has Covariance  │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
m(tau) fit function minimum: -5278.322378861789
m(tau) fit converged: True
m(tau) fit full minimizer information: {'n_eval': 1145, 'n_iter': 1145, 'original': FMin(fval=-5278.322378861789, edm=3.526502868647658e-07, tolerance=0.01, nfcn=114

In [69]:
fig,ax = plt.subplots(figsize=(8,8))
lower, upper = obs.limits
low = lower[-1][0]
high = upper[0][0]
bin_width = 0.01
bins = int(float(high - low)/bin_width)
counts, bin_edges = np.histogram(data_np, bins, range=(low,high))
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
err = np.sqrt(counts)
plt.errorbar(bin_centres, counts, yerr=err, fmt='o', markersize=3, color='black')
x_plot = np.linspace(low, high, num=1000)
y_plot_tot = zfit.run(pdf.pdf(x_plot, norm_range=obs))
plt.plot(x_plot, y_plot_tot*len(data_np)/bins*obs.area(), color='navy', linewidth=2.5, alpha=0.6)
ax.tick_params(axis='both', which='major', labelsize=25)
plt.ylabel("Candidates / (%.3f GeV/$c^2$)" % bin_width,fontsize=25)
plt.xlabel("$m(\mu^\pm \gamma)$ [GeV/$c^2$]",fontsize=25)
plt.xlim(low,high)
ymin, ymax = plt.ylim()
plt.ylim(0,1.05*ymax)
plt.tight_layout()
fig.savefig(loc.PLOTS+"/Tau2MuGamma_fit.pdf")